In [90]:
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go
import scipy
from matplotlib.lines import Line2D
import networkx as nx
from tqdm import tqdm
import matplotlib.colors as mcolors
from sklearn.linear_model import LinearRegression
from scipy.optimize import curve_fit
import psycopg2
import statsmodels.api as sm


axis_label_fontsize = 12
title_fontsize = 15
pd.set_option("display.max_columns",100)

In [91]:
# Load electricity, gas, and fuel prices
df_elec_price = pd.read_csv("data//electricity_prices_SEMESTER_Hungary_EUROSTAT_2007-2024.csv")
df_elec_price = df_elec_price.rename(columns = {"lower_bound_[currency]":"lower_bound_HUF", 
                                        "upper_bound_[currency]":"upper_bound_HUF"})
df_elec_price["semester"] = df_elec_price["semester"].str.replace("-S1","_1").str.replace("-S2","_2")


df_gas_price = pd.read_csv("data/gas_prices_SEMESTER_Hungary_EUROSTAT_2007-2024.csv")
df_gas_price = df_gas_price.rename(columns = {"lower_bound_[currency]":"lower_bound_HUF", 
                                        "upper_bound_[currency]":"upper_bound_HUF"})
df_gas_price["semester"] = df_gas_price["semester"].str.replace("-S1","_1").str.replace("-S2","_2")


df_fuel_price = pd.read_csv("data/fuel_price_Hungary_SEMESTER_2005-2024.csv")



In [92]:

# Load toy dataset
df_semester = pd.read_csv("data/energy_purchases_revenue_firms_2020-2024.csv")

# --------------------------
# Parameters
# --------------------------
enforce_yearly_energy = True
required_years = range(2020, 2025)

# --------------------------
# Step 1: Yearly energy input condition
# --------------------------
if enforce_yearly_energy:
    input_condition = pd.Series(True, index=df_semester.index)
    for year in required_years:
        sem1 = f"{year}_1"
        sem2 = f"{year}_2"

        # Toy dataset columns follow oil_purchases_2020_1, gas_purchases_2020_1, electricity_purchases_2020_1
        elec_sum = df_semester.get(f"electricity_purchases_{sem1}", 0) + df_semester.get(f"electricity_purchases_{sem2}", 0)
        gas_sum  = df_semester.get(f"gas_purchases_{sem1}", 0)        + df_semester.get(f"gas_purchases_{sem2}", 0)

        yearly_condition = (elec_sum > 0) & (gas_sum > 0)
        input_condition &= yearly_condition
else:
    input_condition = pd.Series(True, index=df_semester.index)

# --------------------------
# Step 2: Revenue consistency condition (annual only)
# --------------------------
revenue_condition = (
    (df_semester["revenue_2020"] > 0) &
    (df_semester["revenue_2021"] > 0) &
    (df_semester["revenue_2022"] > 0) &
    (df_semester["revenue_2023"] > 0) &
    (df_semester["revenue_2024"] > 0)
)

# --------------------------
# Step 3: Sector filters
# --------------------------
# Use firm-level NACE values (constant across years in toy dataset)
sector_filter = (
    (df_semester["nace2"] != 35) &
    (df_semester["nace4"] != 4730) &
    (df_semester["nace4"] != 4671) &
    (df_semester["nace4"] != 1920) &
    (df_semester["nace4"] != 610) &
    (df_semester["nace2"] != 64) &
    (df_semester["nace2"] != 65) &
    (df_semester["nace2"] != 66) &
    (df_semester["nace2"] > 0) &
    (df_semester["nace4"] != 5221)
)

# --------------------------
# Step 4: Combine filters
# --------------------------
final_condition = input_condition & revenue_condition & sector_filter

# --------------------------
# Step 5: Apply filters
# --------------------------
df_final = df_semester[final_condition].copy()
# --------------------------
# Step 6: Remove ETS firms
# --------------------------
df_final = df_final[df_final.ets_flag == 0].reset_index(drop=True)

print("Final sample size:", len(df_final))


Final sample size: 77


## Estimate Electricity, Gas and Oil consumption for every firm for every year between 2020-2024 using semi-annual energy prices [for Hungary]

In [93]:
def build_price_lookup(bounds, prices):
    return [(row[0], row[1], price) for row, price in zip(bounds, prices)]

def find_matching_price(val, lookup):
    for lower, upper, price in lookup:
        if lower <= val <= upper:
            return price
    # Fallback: closest bound
    all_bounds = [b for bounds in lookup for b in bounds[:2]]
    print(all_bounds)
    closest_bound = min(all_bounds, key=lambda x: abs(x - val))
    for lower, upper, price in lookup:
        if lower == closest_bound or upper == closest_bound:
            return price
    return np.nan


def compute_energy_from_price(df, df_price, energy_type, semesters):
    """Compute electricity/gas consumption with annual bounds logic.
       Adapted for toy dataset (columns: electricity_purchases_*, gas_purchases_*)."""
    years = sorted(set([s.split("_")[0] for s in semesters]))

    for year in years:
        sem1, sem2 = f"{year}_1", f"{year}_2"

        # Price tables
        df_price_sem1 = df_price[df_price.semester == sem1].reset_index(drop=True)
        df_price_sem2 = df_price[df_price.semester == sem2].reset_index(drop=True)

        bounds = list(zip(df_price_sem1["lower_bound_HUF"], df_price_sem1["upper_bound_HUF"]))
        prices_sem1 = df_price_sem1[f"{energy_type}_price_per_MWh"]
        prices_sem2 = df_price_sem2[f"{energy_type}_price_per_MWh"]

        price_lookup_sem1 = build_price_lookup(bounds, prices_sem1)
        price_lookup_sem2 = build_price_lookup(bounds, prices_sem2)

        # Input value columns in toy dataset
        val_col_sem1 = f"{energy_type}purchases_{sem1}" if energy_type == "oil" else f"{energy_type}tricity_purchases_{sem1}"
        val_col_sem2 = f"{energy_type}purchases_{sem2}" if energy_type == "oil" else f"{energy_type}tricity_purchases_{sem2}"

        if energy_type == "gas":
            val_col_sem1 = f"gas_purchases_{sem1}"
            val_col_sem2 = f"gas_purchases_{sem2}"
        elif energy_type == "elec":
            val_col_sem1 = f"electricity_purchases_{sem1}"
            val_col_sem2 = f"electricity_purchases_{sem2}"
        elif energy_type == "oil":
            val_col_sem1 = f"oil_purchases_{sem1}"
            val_col_sem2 = f"oil_purchases_{sem2}"

        # Output columns
        price_col_sem1 = f"{energy_type}_price_{sem1}"
        price_col_sem2 = f"{energy_type}_price_{sem2}"
        kwh_col_sem1 = f"{energy_type}_consumption_KWH_{sem1}"
        kwh_col_sem2 = f"{energy_type}_consumption_KWH_{sem2}"

        values_sem1 = df[val_col_sem1].fillna(0).values
        values_sem2 = df[val_col_sem2].fillna(0).values

        prices_out_sem1, prices_out_sem2 = [], []
        kwh_out_sem1, kwh_out_sem2 = [], []

        for val1, val2 in zip(values_sem1, values_sem2):
            annual_val = val1 + val2
            if annual_val > 0:
                price1 = find_matching_price(annual_val, price_lookup_sem1)
                price2 = find_matching_price(annual_val, price_lookup_sem2)

                if val1 > 0 and val2 == 0:
                    mean_price = np.nanmean([price1, price2])
                    prices_out_sem1.append(mean_price)
                    prices_out_sem2.append(mean_price)
                    kwh_val = (annual_val / mean_price) * 1000 if mean_price > 0 else 0
                    kwh_out_sem1.append(kwh_val)
                    kwh_out_sem2.append(0)

                elif val2 > 0 and val1 == 0:
                    mean_price = np.nanmean([price1, price2])
                    prices_out_sem1.append(mean_price)
                    prices_out_sem2.append(mean_price)
                    kwh_val = (annual_val / mean_price) * 1000 if mean_price > 0 else 0
                    kwh_out_sem1.append(0)
                    kwh_out_sem2.append(kwh_val)

                else:
                    prices_out_sem1.append(price1)
                    prices_out_sem2.append(price2)
                    kwh_out_sem1.append((val1 / price1) * 1000 if price1 > 0 and val1 > 0 else 0)
                    kwh_out_sem2.append((val2 / price2) * 1000 if price2 > 0 and val2 > 0 else 0)

            else:
                prices_out_sem1.append(0)
                prices_out_sem2.append(0)
                kwh_out_sem1.append(0)
                kwh_out_sem2.append(0)

        df[price_col_sem1] = prices_out_sem1
        df[price_col_sem2] = prices_out_sem2
        df[kwh_col_sem1] = kwh_out_sem1
        df[kwh_col_sem2] = kwh_out_sem2

    return df


def compute_oil_consumption(df, df_fuel_price, semesters, conversion_factor=9.6):
    """Compute oil consumption, handling annual reporting if only one semester is filled.
       Adapted for toy dataset (columns: oil_purchases_*)."""
    years = sorted(set([s.split("_")[0] for s in semesters]))

    for year in years:
        sem1, sem2 = f"{year}_1", f"{year}_2"

        price_row1 = df_fuel_price[df_fuel_price['year_semester'] == sem1]
        price_row2 = df_fuel_price[df_fuel_price['year_semester'] == sem2]

        oil_price1 = price_row1['fuel_price_HUF'].values[0] if not price_row1.empty else np.nan
        oil_price2 = price_row2['fuel_price_HUF'].values[0] if not price_row2.empty else np.nan

        oil_col_sem1 = f"oil_purchases_{sem1}"
        oil_col_sem2 = f"oil_purchases_{sem2}"

        price_col_sem1 = f"oil_price_{sem1}"
        price_col_sem2 = f"oil_price_{sem2}"
        kwh_col_sem1 = f"oil_consumption_KWH_{sem1}"
        kwh_col_sem2 = f"oil_consumption_KWH_{sem2}"

        values_sem1 = df[oil_col_sem1].fillna(0).values if oil_col_sem1 in df.columns else np.zeros(len(df))
        values_sem2 = df[oil_col_sem2].fillna(0).values if oil_col_sem2 in df.columns else np.zeros(len(df))

        prices_out_sem1, prices_out_sem2 = [], []
        kwh_out_sem1, kwh_out_sem2 = [], []

        for val1, val2 in zip(values_sem1, values_sem2):
            annual_val = val1 + val2

            if annual_val > 0:
                if val1 > 0 and val2 == 0:
                    mean_price = np.nanmean([oil_price1, oil_price2])
                    prices_out_sem1.append(mean_price)
                    prices_out_sem2.append(mean_price)
                    kwh_val = (annual_val / mean_price) * conversion_factor if mean_price > 0 else 0
                    kwh_out_sem1.append(kwh_val)
                    kwh_out_sem2.append(0)

                elif val2 > 0 and val1 == 0:
                    mean_price = np.nanmean([oil_price1, oil_price2])
                    prices_out_sem1.append(mean_price)
                    prices_out_sem2.append(mean_price)
                    kwh_val = (annual_val / mean_price) * conversion_factor if mean_price > 0 else 0
                    kwh_out_sem1.append(0)
                    kwh_out_sem2.append(kwh_val)

                else:
                    prices_out_sem1.append(oil_price1)
                    prices_out_sem2.append(oil_price2)
                    kwh_out_sem1.append((val1 / oil_price1) * conversion_factor if oil_price1 > 0 and val1 > 0 else 0)
                    kwh_out_sem2.append((val2 / oil_price2) * conversion_factor if oil_price2 > 0 and val2 > 0 else 0)

            else:
                prices_out_sem1.append(0)
                prices_out_sem2.append(0)
                kwh_out_sem1.append(0)
                kwh_out_sem2.append(0)

        df[price_col_sem1] = prices_out_sem1
        df[price_col_sem2] = prices_out_sem2
        df[kwh_col_sem1] = kwh_out_sem1
        df[kwh_col_sem2] = kwh_out_sem2

    return df


# --------------------------
# Run on toy dataset
# --------------------------
years = np.arange(2020, 2025)
semesters = [f"{year}_1" for year in years] + [f"{year}_2" for year in years]

df_final = compute_energy_from_price(df_final, df_elec_price, "elec", semesters)
df_final = compute_energy_from_price(df_final, df_gas_price, "gas", semesters)
df_final = compute_oil_consumption(df_final, df_fuel_price, semesters)

# Annual consumption totals
for year in years:
    elec_cols = [f"elec_consumption_KWH_{year}_1", f"elec_consumption_KWH_{year}_2"]
    gas_cols  = [f"gas_consumption_KWH_{year}_1", f"gas_consumption_KWH_{year}_2"]
    oil_cols  = [f"oil_consumption_KWH_{year}_1", f"oil_consumption_KWH_{year}_2"]

    df_final[f"elec_consumption_KWH_{year}"] = df_final[elec_cols].sum(axis=1, skipna=True)
    df_final[f"gas_consumption_KWH_{year}"]  = df_final[gas_cols].sum(axis=1, skipna=True)
    df_final[f"oil_consumption_KWH_{year}"]  = df_final[oil_cols].sum(axis=1, skipna=True)


[7209891.012, 72091123.5, 76325061.06, 763242917.04, 829356219.04, 8293553830.495999, 8203034340.2, 32812129092.147995, 49994595551.2, inf, 0.0, 4764232.557999999]
[7209891.012, 72091123.5, 76325061.06, 763242917.04, 829356219.04, 8293553830.495999, 8203034340.2, 32812129092.147995, 49994595551.2, inf, 0.0, 4764232.557999999]
[17554584.88, 175526890.0, 144590115.67200002, 1445886582.048, 1314356607.04, 13143552821.696, 22351101214.2, 89404382326.90799, 36742918283.200005, inf, 0.0, 14748729.099999998]
[17554584.88, 175526890.0, 144590115.67200002, 1445886582.048, 1314356607.04, 13143552821.696, 22351101214.2, 89404382326.90799, 36742918283.200005, inf, 0.0, 14748729.099999998]
[17554584.88, 175526890.0, 144590115.67200002, 1445886582.048, 1314356607.04, 13143552821.696, 22351101214.2, 89404382326.90799, 36742918283.200005, inf, 0.0, 14748729.099999998]
[17554584.88, 175526890.0, 144590115.67200002, 1445886582.048, 1314356607.04, 13143552821.696, 22351101214.2, 89404382326.90799, 367429

C:\Users\Johannes\AppData\Local\Temp\ipykernel_26780\2523089237.py:138: RuntimeWarning: Mean of empty slice
  mean_price = np.nanmean([oil_price1, oil_price2])
C:\Users\Johannes\AppData\Local\Temp\ipykernel_26780\2523089237.py:146: RuntimeWarning: Mean of empty slice
  mean_price = np.nanmean([oil_price1, oil_price2])
C:\Users\Johannes\AppData\Local\Temp\ipykernel_26780\2523089237.py:138: RuntimeWarning: Mean of empty slice
  mean_price = np.nanmean([oil_price1, oil_price2])
C:\Users\Johannes\AppData\Local\Temp\ipykernel_26780\2523089237.py:146: RuntimeWarning: Mean of empty slice
  mean_price = np.nanmean([oil_price1, oil_price2])
C:\Users\Johannes\AppData\Local\Temp\ipykernel_26780\2523089237.py:138: RuntimeWarning: Mean of empty slice
  mean_price = np.nanmean([oil_price1, oil_price2])
C:\Users\Johannes\AppData\Local\Temp\ipykernel_26780\2523089237.py:146: RuntimeWarning: Mean of empty slice
  mean_price = np.nanmean([oil_price1, oil_price2])
C:\Users\Johannes\AppData\Local\Temp\ipy

In [94]:
## Calculate electricity share for every year 2020-2024

df_final["elec_share_2020"] = df_final["elec_consumption_KWH_2020"]/(df_final["elec_consumption_KWH_2020"] + df_final["gas_consumption_KWH_2020"] + df_final["oil_consumption_KWH_2020"])
df_final["elec_share_2021"] = df_final["elec_consumption_KWH_2021"]/(df_final["elec_consumption_KWH_2021"] + df_final["gas_consumption_KWH_2021"] + df_final["oil_consumption_KWH_2021"])
df_final["elec_share_2022"] = df_final["elec_consumption_KWH_2022"]/(df_final["elec_consumption_KWH_2022"] + df_final["gas_consumption_KWH_2022"] + df_final["oil_consumption_KWH_2022"])
df_final["elec_share_2023"] = df_final["elec_consumption_KWH_2023"]/(df_final["elec_consumption_KWH_2023"] + df_final["gas_consumption_KWH_2023"] + df_final["oil_consumption_KWH_2023"])
df_final["elec_share_2024"] = df_final["elec_consumption_KWH_2024"]/(df_final["elec_consumption_KWH_2024"] + df_final["gas_consumption_KWH_2024"] + df_final["oil_consumption_KWH_2024"])
df_final

,firm_id,oil_purchases_2020_1,gas_purchases_2020_1,electricity_purchases_2020_1,oil_purchases_2020_2,gas_purchases_2020_2,electricity_purchases_2020_2,oil_purchases_2021_1,gas_purchases_2021_1,electricity_purchases_2021_1,oil_purchases_2021_2,gas_purchases_2021_2,electricity_purchases_2021_2,oil_purchases_2022_1,gas_purchases_2022_1,electricity_purchases_2022_1,oil_purchases_2022_2,gas_purchases_2022_2,electricity_purchases_2022_2,oil_purchases_2023_1,gas_purchases_2023_1,electricity_purchases_2023_1,oil_purchases_2023_2,gas_purchases_2023_2,electricity_purchases_2023_2,oil_purchases_2024_1,gas_purchases_2024_1,electricity_purchases_2024_1,oil_purchases_2024_2,gas_purchases_2024_2,electricity_purchases_2024_2,revenue_2020,revenue_2021,revenue_2022,revenue_2023,revenue_2024,employment_2020,employment_2021,employment_2022,employment_2023,employment_2024,ets_flag,nace2,nace4,parent_nace,fossil_purchases_2020,fossil_purchases_2021,fossil_purchases_2022,fossil_purchases_2023,fossil_purchases_2024,...,gas_consumption_KWH_2022_1,gas_consumption_KWH_2022_2,gas_price_2023_1,gas_price_2023_2,gas_consumption_KWH_2023_1,gas_consumption_KWH_2023_2,gas_price_2024_1,gas_price_2024_2,gas_consumption_KWH_2024_1,gas_consumption_KWH_2024_2,oil_price_2020_1,oil_price_2020_2,oil_consumption_KWH_2020_1,oil_consumption_KWH_2020_2,oil_price_2021_1,oil_price_2021_2,oil_consumption_KWH_2021_1,oil_consumption_KWH_2021_2,oil_price_2022_1,oil_price_2022_2,oil_consumption_KWH_2022_1,oil_consumption_KWH_2022_2,oil_price_2023_1,oil_price_2023_2,oil_consumption_KWH_2023_1,oil_consumption_KWH_2023_2,oil_price_2024_1,oil_price_2024_2,oil_consumption_KWH_2024_1,oil_consumption_KWH_2024_2,elec_consumption_KWH_2020,gas_consumption_KWH_2020,oil_consumption_KWH_2020,elec_consumption_KWH_2021,gas_consumption_KWH_2021,oil_consumption_KWH_2021,elec_consumption_KWH_2022,gas_consumption_KWH_2022,oil_consumption_KWH_2022,elec_consumption_KWH_2023,gas_consumption_KWH_2023,oil_consumption_KWH_2023,elec_consumption_KWH_2024,gas_consumption_KWH_2024,oil_consumption_KWH_2024,elec_share_2020,elec_share_2021,elec_share_2022,elec_share_2023,elec_share_2024
0,4,7449272,439993,499757,0,489835,287877,0,380148,7679711,0,348533,7769324,0,498759,6486856,0,312647,454676,6328985,345413,173227,0,382332,396784,0,111585,323459,0,449807,8989317,94803667686,94580321673,97283053944,96246309501,97880364090,412,432,433,449,449,0,6,692,B,8379100,728681,811406,7056730,561392,...,29080.292226,8651.762193,53095.0,45810.4,6505.565496,8345.965108,40320.5,40060.1,2767.450800,11228.304473,NaN,NaN,0,0,0.0,0.0,0,0,0.0,0.0,0,0,NaN,NaN,0,0,0.0,0.0,0,0,13780.435958,72931.464833,0,301954.493840,52746.792569,0,84018.699990,37732.054419,0,4440.695457,14851.530604,0,71702.818879,13995.755273,0,0.158922,0.851292,0.690088,0.230181,0.836686
1,5,0,8321647,496203,0,317022,199286,8577523,398794,317298,0,401356,6555250,6503672,233119,116050,7938448,473480,114882,8465168,456533,443013,0,174689,177948,0,363260,449087,0,404908,105237,29760229063,29601583083,29987849022,30452737001,29741905508,278,273,264,269,277,0,6,692,B,8638669,9377673,15148719,9096390,768168,...,13592.072812,13102.432977,53095.0,45810.4,8598.417930,3813.304402,40320.5,40060.1,9009.312881,10107.513461,0.0,0.0,0,0,NaN,NaN,0,0,NaN,NaN,0,0,NaN,NaN,0,0,0.0,0.0,0,0,12178.538820,694124.130337,0,129035.945197,57594.060269,0,2359.643342,26694.505789,0,4458.337221,12411.722333,0,4975.455089,19116.826342,0,0.017243,0.691400,0.081215,0.264275,0.206517
2,6,0,339021,236997,0,491869,480491,0,220601,421141,0,498764,6955905,0,479775,9108484,0,204541,399475,0,8483625,370889,0,345545,310487,0,270993,9231882,0,416133,427586,50972429533,53126838082,54335710004,54730965958,56249601499,107,109,110,114,120,0,16,1691,C,830890,719365,684316,8829170,687126,...,27973.424445,5660.185739,53095.0,45810.4,159781.994538,7542.937848,40320.5,40060.1,6720.973202,10387.717455,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,12512.340527,65206.518912,0,138645.660898,49365.793998

## Calculate low-carbon electricity share of every firm

In [95]:
## low-carbon share of the Hungarian electricity mix, derived from https://ember-energy.org/data/electricity-data-explorer/
df_lowcarbon_share_Hungary = pd.read_csv("data/low-carbon_electricity_share_Hungary_[2014-2024].csv")
df_lowcarbon_share_Hungary


df_final["total_consumption_KWH_2020"] = (df_final["elec_consumption_KWH_2020"] + df_final["gas_consumption_KWH_2020"] + df_final["oil_consumption_KWH_2020"])
df_final["total_consumption_KWH_2021"] = (df_final["elec_consumption_KWH_2021"] + df_final["gas_consumption_KWH_2021"] + df_final["oil_consumption_KWH_2021"])
df_final["total_consumption_KWH_2022"] = (df_final["elec_consumption_KWH_2022"] + df_final["gas_consumption_KWH_2022"] + df_final["oil_consumption_KWH_2022"])
df_final["total_consumption_KWH_2023"] = (df_final["elec_consumption_KWH_2023"] + df_final["gas_consumption_KWH_2023"] + df_final["oil_consumption_KWH_2023"])
df_final["total_consumption_KWH_2024"] = (df_final["elec_consumption_KWH_2024"] + df_final["gas_consumption_KWH_2024"] + df_final["oil_consumption_KWH_2024"])
df_final["avg_total_consumption_KWH"] = 1/5*(df_final["total_consumption_KWH_2020"] + df_final["total_consumption_KWH_2021"] + df_final["total_consumption_KWH_2022"] + df_final["total_consumption_KWH_2023"] + df_final["total_consumption_KWH_2024"])

df_final["low-carbon_elec_consumption_KWH_2020"] = df_final["elec_consumption_KWH_2020"] * df_lowcarbon_share_Hungary[df_lowcarbon_share_Hungary.year==2020]["low-carbon_elec_share"].values[0]
df_final["low-carbon_elec_consumption_KWH_2021"] = df_final["elec_consumption_KWH_2021"] * df_lowcarbon_share_Hungary[df_lowcarbon_share_Hungary.year==2021]["low-carbon_elec_share"].values[0]
df_final["low-carbon_elec_consumption_KWH_2022"] = df_final["elec_consumption_KWH_2022"] * df_lowcarbon_share_Hungary[df_lowcarbon_share_Hungary.year==2022]["low-carbon_elec_share"].values[0]
df_final["low-carbon_elec_consumption_KWH_2023"] = df_final["elec_consumption_KWH_2023"] * df_lowcarbon_share_Hungary[df_lowcarbon_share_Hungary.year==2023]["low-carbon_elec_share"].values[0]
df_final["low-carbon_elec_consumption_KWH_2024"] = df_final["elec_consumption_KWH_2024"] * df_lowcarbon_share_Hungary[df_lowcarbon_share_Hungary.year==2024]["low-carbon_elec_share"].values[0]

df_final["low-carbon_elec_share_2020"] = df_final["low-carbon_elec_consumption_KWH_2020"]/(df_final["elec_consumption_KWH_2020"] + df_final["gas_consumption_KWH_2020"] + df_final["oil_consumption_KWH_2020"])
df_final["low-carbon_elec_share_2021"] = df_final["low-carbon_elec_consumption_KWH_2021"]/(df_final["elec_consumption_KWH_2021"] + df_final["gas_consumption_KWH_2021"] + df_final["oil_consumption_KWH_2021"])
df_final["low-carbon_elec_share_2022"] = df_final["low-carbon_elec_consumption_KWH_2022"]/(df_final["elec_consumption_KWH_2022"] + df_final["gas_consumption_KWH_2022"] + df_final["oil_consumption_KWH_2022"])
df_final["low-carbon_elec_share_2023"] = df_final["low-carbon_elec_consumption_KWH_2023"]/(df_final["elec_consumption_KWH_2023"] + df_final["gas_consumption_KWH_2023"] + df_final["oil_consumption_KWH_2023"])
df_final["low-carbon_elec_share_2024"] = df_final["low-carbon_elec_consumption_KWH_2024"]/(df_final["elec_consumption_KWH_2024"] + df_final["gas_consumption_KWH_2024"] + df_final["oil_consumption_KWH_2024"])

df_final["fossil_share_2020"] = 1-df_final["low-carbon_elec_share_2020"]
df_final["fossil_share_2021"] = 1-df_final["low-carbon_elec_share_2021"]
df_final["fossil_share_2022"] = 1-df_final["low-carbon_elec_share_2022"]
df_final["fossil_share_2023"] = 1-df_final["low-carbon_elec_share_2023"]
df_final["fossil_share_2024"] = 1-df_final["low-carbon_elec_share_2024"]

C:\Users\Johannes\AppData\Local\Temp\ipykernel_26780\2816890614.py:27: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final["fossil_share_2022"] = 1-df_final["low-carbon_elec_share_2022"]
C:\Users\Johannes\AppData\Local\Temp\ipykernel_26780\2816890614.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final["fossil_share_2023"] = 1-df_final["low-carbon_elec_share_2023"]
C:\Users\Johannes\AppData\Local\Temp\ipykernel_26780\2816890614.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of 

In [96]:
# --- Employment ---
df_final["avg_employment"] = df_final[[
    "employment_2020", "employment_2021", "employment_2022", "employment_2023", "employment_2024"
]].mean(axis=1)

# --- Low-carbon electricity ---
df_final["avg_lowcarbon_elec_consumption_KWH"] = df_final[[
    "low-carbon_elec_consumption_KWH_2020",
    "low-carbon_elec_consumption_KWH_2021",
    "low-carbon_elec_consumption_KWH_2022",
    "low-carbon_elec_consumption_KWH_2023",
    "low-carbon_elec_consumption_KWH_2024"
]].mean(axis=1)

df_final["avg_lowcarbon_elec_share"] = df_final[[
    "low-carbon_elec_share_2020",
    "low-carbon_elec_share_2021",
    "low-carbon_elec_share_2022",
    "low-carbon_elec_share_2023",
    "low-carbon_elec_share_2024"
]].mean(axis=1)

# --- Revenue ---
df_final["avg_revenue"] = df_final[[
    "revenue_2020", "revenue_2021", "revenue_2022", "revenue_2023", "revenue_2024"
]].mean(axis=1)

df_final["avg_revenue_per_energy"] = df_final["avg_revenue"] / df_final["avg_total_consumption_KWH"]

# --- Electricity cost (proxy = purchases) ---
df_final["avg_elec_cost"] = df_final[[
    "electricity_purchases_2020_1", "electricity_purchases_2020_2",
    "electricity_purchases_2021_1", "electricity_purchases_2021_2",
    "electricity_purchases_2022_1", "electricity_purchases_2022_2",
    "electricity_purchases_2023_1", "electricity_purchases_2023_2",
    "electricity_purchases_2024_1", "electricity_purchases_2024_2"
]].mean(axis=1)

# --- Fossil cost (proxy = oil + gas purchases) ---
fossil_cols = [c for c in df_final.columns if c.startswith("oil_purchases_") or c.startswith("gas_purchases_")]
df_final["avg_fossil_cost"] = df_final[fossil_cols].mean(axis=1)

# --- Electricity consumption ---
df_final["avg_elec_consumption_KWH"] = df_final[[
    "elec_consumption_KWH_2020",
    "elec_consumption_KWH_2021",
    "elec_consumption_KWH_2022",
    "elec_consumption_KWH_2023",
    "elec_consumption_KWH_2024"
]].mean(axis=1)

# --- Fossil consumption (proxy = oil + gas consumption) ---
fossil_cons_cols = [c for c in df_final.columns if c.startswith("oil_consumption_KWH_") or c.startswith("gas_consumption_KWH_")]
df_final["avg_fossil_consumption_KWH"] = df_final[fossil_cons_cols].mean(axis=1)


C:\Users\Johannes\AppData\Local\Temp\ipykernel_26780\2358403230.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final["avg_employment"] = df_final[[
C:\Users\Johannes\AppData\Local\Temp\ipykernel_26780\2358403230.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final["avg_lowcarbon_elec_consumption_KWH"] = df_final[[
C:\Users\Johannes\AppData\Local\Temp\ipykernel_26780\2358403230.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poo

## Estimate Decarbonization Trend

In [97]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from tqdm import tqdm

# --- Function to estimate linear trend with SE ---
def estimate_trend_linear(df_temp, index, years_to_use, prefix, robust=False):
    """
    Generic function to estimate linear trend for any prefix (e.g., elec_share, low-carbon_elec_share, fossil_share).
    """
    cols = [f"{prefix}_{y}" for y in years_to_use]
    vals = df_temp.loc[index, cols].astype(float).values

    # sequential indices relative to first year
    t = np.array([y - years_to_use[0] for y in years_to_use]).reshape(-1, 1)

    if np.isnan(vals).any():
        return np.nan, np.nan, np.nan

    t_with_const = sm.add_constant(t)

    if robust:
        model = sm.RLM(vals, t_with_const, M=sm.robust.norms.HuberT())
        results = model.fit()
    else:
        model = sm.OLS(vals, t_with_const)
        results = model.fit()

    intercept, slope = results.params
    slope_se = results.bse[1]

    return slope, intercept, slope_se


# --- Pipeline for one prefix ---
def compute_slopes_linear(df_final, prefix, years):
    slope_all = {"ols": [], "robust": []}
    intercept_all = {"ols": [], "robust": []}
    slope_se_all = {"ols": [], "robust": []}
    slope_leave_one = {y: {"ols": [], "robust": []} for y in years}
    intercept_leave_one = {y: {"ols": [], "robust": []} for y in years}
    values = {y: [] for y in years}

    for idx in tqdm(df_final.index):
        for y in years:
            values[y].append(df_final.loc[idx, f"{prefix}_{y}"])

        b_ols, a_ols, se_ols = estimate_trend_linear(df_final, idx, years, prefix, robust=False)
        b_robust, a_robust, se_robust = estimate_trend_linear(df_final, idx, years, prefix, robust=True)

        for key, val in zip(["ols", "robust"], [(b_ols, a_ols, se_ols), (b_robust, a_robust, se_robust)]):
            slope_all[key].append(val[0])
            intercept_all[key].append(val[1])
            slope_se_all[key].append(val[2])

        for omit_year in years:
            yrs_sub = [y for y in years if y != omit_year]
            b_ols_omit, a_ols_omit, se_ols_omit = estimate_trend_linear(df_final, idx, yrs_sub, prefix, robust=False)
            b_robust_omit, a_robust_omit, se_robust_omit = estimate_trend_linear(df_final, idx, yrs_sub, prefix, robust=True)
            slope_leave_one[omit_year]["ols"].append(b_ols_omit)
            intercept_leave_one[omit_year]["ols"].append(a_ols_omit)
            slope_leave_one[omit_year]["robust"].append(b_robust_omit)
            intercept_leave_one[omit_year]["robust"].append(a_robust_omit)

    # Build record rows
    records = []
    for i in range(len(df_final)):
        for omit_year in years:
            rec = {"firm_id": i, "omit_year": omit_year}
            for y in years:
                rec[f"{prefix}_{y}"] = values[y][i]

            for kind in ["ols", "robust"]:
                rec[f"{prefix}_slope_all_{kind}"] = slope_all[kind][i]
                rec[f"{prefix}_intercept_all_{kind}"] = intercept_all[kind][i]
                rec[f"{prefix}_slope_se_{kind}"] = slope_se_all[kind][i]
                rec[f"{prefix}_slope_omit_{kind}"] = slope_leave_one[omit_year][kind][i]
                rec[f"{prefix}_intercept_omit_{kind}"] = intercept_leave_one[omit_year][kind][i]
                rec[f"{prefix}_delta_{kind}"] = rec[f"{prefix}_slope_omit_{kind}"] - rec[f"{prefix}_slope_all_{kind}"]
                rec[f"{prefix}_rel_delta_pct_{kind}"] = (
                    100 * rec[f"{prefix}_delta_{kind}"] /
                    (np.abs(rec[f"{prefix}_slope_all_{kind}"]) if rec[f"{prefix}_slope_all_{kind}"] != 0 else 1)
                )

            records.append(rec)

    return pd.DataFrame(records)


# --- Run for toy dataset ---
years = [2020, 2021, 2022, 2023, 2024]

df_elec = compute_slopes_linear(df_final, prefix="elec_share", years=years)
df_lowcarb = compute_slopes_linear(df_final, prefix="low-carbon_elec_share", years=years)

# Merge results side by side
df_slopes_linear = df_elec.merge(df_lowcarb, on=["firm_id", "omit_year"], how="inner")


100%|██████████| 77/77 [00:03<00:00, 24.12it/s]


## Estimate Decarbonization Rate

In [98]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from tqdm import tqdm

# --- Function to estimate exponential growth with SE ---
def estimate_growth_exp(df_temp, index, years_to_use, prefix, robust=False):
    """
    Estimate exponential growth model for any prefix (log(y) = a + b*t).
    Returns: slope (growth rate), intercept (log-initial), slope_se
    """
    cols = [f"{prefix}_{y}" for y in years_to_use]
    vals = df_temp.loc[index, cols].astype(float).values

    # sequential time index
    t = np.array([y - years_to_use[0] for y in years_to_use]).reshape(-1, 1)

    # log-transform safely
    vals = np.where(vals <= 0, np.nan, vals)
    if np.isnan(vals).any():
        return np.nan, np.nan, np.nan

    y_log = np.log(vals)
    t_with_const = sm.add_constant(t)

    if robust:
        model = sm.RLM(y_log, t_with_const, M=sm.robust.norms.HuberT())
        results = model.fit()
    else:
        model = sm.OLS(y_log, t_with_const)
        results = model.fit()

    intercept, slope = results.params
    slope_se = results.bse[1]
    return slope, intercept, slope_se


# --- Pipeline for one prefix ---
def compute_growth_exp(df_final, prefix, years):
    slope_all = {"ols": [], "robust": []}
    intercept_all = {"ols": [], "robust": []}
    slope_se_all = {"ols": [], "robust": []}
    slope_leave_one = {y: {"ols": [], "robust": []} for y in years}
    intercept_leave_one = {y: {"ols": [], "robust": []} for y in years}
    values = {y: [] for y in years}

    for idx in tqdm(df_final.index):
        for y in years:
            values[y].append(df_final.loc[idx, f"{prefix}_{y}"])

        b_ols, a_ols, se_ols = estimate_growth_exp(df_final, idx, years, prefix=prefix, robust=False)
        b_robust, a_robust, se_robust = estimate_growth_exp(df_final, idx, years, prefix=prefix, robust=True)

        for key, val in zip(["ols", "robust"], [(b_ols, a_ols, se_ols), (b_robust, a_robust, se_robust)]):
            slope_all[key].append(val[0])
            intercept_all[key].append(val[1])
            slope_se_all[key].append(val[2])

        for omit_year in years:
            yrs_sub = [y for y in years if y != omit_year]
            b_ols_omit, a_ols_omit, _ = estimate_growth_exp(df_final, idx, yrs_sub, prefix=prefix, robust=False)
            b_robust_omit, a_robust_omit, _ = estimate_growth_exp(df_final, idx, yrs_sub, prefix=prefix, robust=True)
            slope_leave_one[omit_year]["ols"].append(b_ols_omit)
            intercept_leave_one[omit_year]["ols"].append(a_ols_omit)
            slope_leave_one[omit_year]["robust"].append(b_robust_omit)
            intercept_leave_one[omit_year]["robust"].append(a_robust_omit)

    # Build record rows
    records = []
    for i in range(len(df_final)):
        for omit_year in years:
            rec = {"firm_id": i, "omit_year": omit_year}
            for y in years:
                rec[f"{prefix}_{y}"] = values[y][i]

            for kind in ["ols", "robust"]:
                rec[f"{prefix}_slope_all_{kind}"] = slope_all[kind][i]
                rec[f"{prefix}_intercept_all_{kind}"] = intercept_all[kind][i]
                rec[f"{prefix}_slope_se_{kind}"] = slope_se_all[kind][i]
                rec[f"{prefix}_slope_omit_{kind}"] = slope_leave_one[omit_year][kind][i]
                rec[f"{prefix}_intercept_omit_{kind}"] = intercept_leave_one[omit_year][kind][i]
                rec[f"{prefix}_delta_{kind}"] = rec[f"{prefix}_slope_omit_{kind}"] - rec[f"{prefix}_slope_all_{kind}"]
                rec[f"{prefix}_rel_delta_pct_{kind}"] = (
                    100 * rec[f"{prefix}_delta_{kind}"] /
                    (np.abs(rec[f"{prefix}_slope_all_{kind}"]) if rec[f"{prefix}_slope_all_{kind}"] != 0 else 1)
                )

            records.append(rec)

    return pd.DataFrame(records)


# --- Run for toy dataset ---
years = [2020, 2021, 2022, 2023, 2024]

df_elec_exp = compute_growth_exp(df_final, prefix="elec_share", years=years)
df_lowcarb_exp = compute_growth_exp(df_final, prefix="low-carbon_elec_share", years=years)

# Merge on id + omit_year
df_slopes_exp = pd.merge(df_elec_exp, df_lowcarb_exp, on=["firm_id", "omit_year"], how="inner")


100%|██████████| 77/77 [00:10<00:00,  7.15it/s]


In [99]:
df_slopes_exp

,firm_id,omit_year,elec_share_2020,elec_share_2021,elec_share_2022,elec_share_2023,elec_share_2024,elec_share_slope_all_ols,elec_share_intercept_all_ols,elec_share_slope_se_ols,elec_share_slope_omit_ols,elec_share_intercept_omit_ols,elec_share_delta_ols,elec_share_rel_delta_pct_ols,elec_share_slope_all_robust,elec_share_intercept_all_robust,elec_share_slope_se_robust,elec_share_slope_omit_robust,elec_share_intercept_omit_robust,elec_share_delta_robust,elec_share_rel_delta_pct_robust,low-carbon_elec_share_2020,low-carbon_elec_share_2021,low-carbon_elec_share_2022,low-carbon_elec_share_2023,low-carbon_elec_share_2024,low-carbon_elec_share_slope_all_ols,low-carbon_elec_share_intercept_all_ols,low-carbon_elec_share_slope_se_ols,low-carbon_elec_share_slope_omit_ols,low-carbon_elec_share_intercept_omit_ols,low-carbon_elec_share_delta_ols,low-carbon_elec_share_rel_delta_pct_ols,low-carbon_elec_share_slope_all_robust,low-carbon_elec_share_intercept_all_robust,low-carbon_elec_share_slope_se_robust,low-carbon_elec_share_slope_omit_robust,low-carbon_elec_share_intercept_omit_robust,low-carbon_elec_share_delta_robust,low-carbon_elec_share_rel_delta_pct_robust
0,0,2020,0.158922,0.851292,0.690088,0.230181,0.836686,0.201418,-1.206531,0.264626,-0.114987,-0.372302,-3.164053e-01,-1.570890e+02,0.201418,-1.206531,0.264626,-0.056138,-0.313453,-2.575562e-01,-1.278716e+02,0.098211,0.540016,0.452665,0.163822,0.617325,0.248373,-1.700891,0.260227,-0.061499,-0.832775,-3.098715e-01,-1.247606e+02,0.248373,-1.700891,0.260227,-0.009935,-0.781212,-2.583077e-01,-1.040000e+02
1,0,2021,0.158922,0.851292,0.690088,0.230181,0.836686,0.201418,-1.206531,0.264626,0.322005,-1.688881,1.205876e-01,5.986936e+01,0.201418,-1.206531,0.264626,0.322005,-1.688881,1.205876e-01,5.986936e+01,0.098211,0.540016,0.452665,0.163822,0.617325,0.248373,-1.700891,0.260227,0.367853,-2.178813,1.194804e-01,4.810523e+01,0.248373,-1.700891,0.260227,0.367853,-2.178813,1.194804e-01,4.810523e+01
2,0,2022,0.158922,0.851292,0.690088,0.230181,0.836686,0.201418,-1.206531,0.264626,0.201418,-1.314720,2.775558e-17,1.378010e-14,0.201418,-1.206531,0.264626,0.201418,-1.314720,2.775558e-17,1.378010e-14,0.098211,0.540016,0.452665,0.163822,0.617325,0.248373,-1.700891,0.260227,0.248373,-1.803777,8.326673e-17,3.352488e-14,0.248373,-1.700891,0.260227,0.248373,-1.803777,8.326673e-17,3.352488e-14
3,0,2023,0.158922,0.851292,0.690088,0.230181,0.836686,0.201418,-1.206531,0.264626,0.325220,-1.206531,1.238020e-01,6.146526e+01,0.201418,-1.206531,0.264626,0.325220,-1.206531,1.238020e-01,6.146526e+01,0.098211,0.540016,0.452665,0.163822,0.617325,0.248373,-1.700891,0.260227,0.370259,-1.700891,1.218859e-01,4.907374e+01,0.248373,-1.700891,0.260227,0.370259,-1.700891,1.218859e-01,4.907374e+01
4,0,2024,0.158922,0.851292,0.690088,0.230181,0.836686,0.201418,-1.206531,0.264626,0.090141,-1.095254,-1.112766e-01,-5.524661e+01,0.201418,-1.206531,0.264626,0.090141,-1.095254,-1.112766e-01,-5.524661e+01,0.098211,0.540016,0.452665,0.163822,0.617325,0.248373,-1.700891,0.260227,0.135853,-1.588372,-1.125195e-01,-4.530264e+01,0.248373,-1.700891,0.260227,0.135853,-1.588372,-1.125195e-01,-4.530264e+01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,76,2020,0.837844,0.030055,0.787541,0.034649,0.020712,-0.725801,-0.780403,0.533297,-0.424062,-2.109683,3.017397e-01,4.157332e+01,-0.887960,-0.400246,0.355859,-0.220441,-2.924166,6.675188e-01,7.517444e+01,0.517774,0.019065,0.516590,0.024660,0.015282,-0.678846,-1.274763,0.531771,-0.370573,-2.570156,3.082735e-01,4.541139e+01,-0.844398,-0.877033,0.348679,-0.173611,-3.358002,6.707869e-01,7.943964e+01
381,76,2021,0.837844,0.030055,0.787541,0.034649,0.020712,-0.725801,-0.780403,0.533297,-1.011305,0.361613,-2.855038e-01,-3.933636e+01,-0.887960,-0.400246,0.355859,-0.983375,0.054385,-9.541552e-02,-1.074548e+01,0.517774,0.019065,0.516590,0.024660,0.015282,-0.678846,-1.274763,0.531771,-0.965457,

In [100]:
# --- Function to merge slopes (OLS + robust) for linear or exponential slopes ---
def merge_slopes_all(df_final, df_slopes, prefix, rename_prefix=None):
    """
    Merge slope estimates (OLS and robust) from df_slopes into df_final.

    Parameters:
    - df_final: main DataFrame
    - df_slopes: slope DataFrame (linear or exponential)
    - prefix: prefix used in df_slopes (e.g., 'elec_share', 'low-carbon_elec_share')
    - rename_prefix: prefix to use in df_final (e.g., 'elec_share_exp')
    """
    if rename_prefix is None:
        rename_prefix = prefix

    omit_years = sorted(df_slopes["omit_year"].unique())

    for slope_type in ["ols", "robust"]:
        slope_col = f"{prefix}_slope_all_{slope_type}"
        intercept_col = f"{prefix}_intercept_all_{slope_type}"
        se_col = f"{prefix}_slope_se_{slope_type}"

        # Full-sample slopes
        df_all = df_slopes.groupby("firm_id").last().reset_index()
        df_final = df_final.merge(
            df_all[["firm_id", slope_col, intercept_col, se_col]],
            on="firm_id", how="left"
        ).rename(columns={
            slope_col: f"{rename_prefix}_growth_{slope_type}",
            intercept_col: f"{rename_prefix}_intercept_{slope_type}",
            se_col: f"{rename_prefix}_growth_se_{slope_type}"
        })

        # Leave-one-year-out slopes
        for y in omit_years:
            df_omit = df_slopes[df_slopes["omit_year"] == y].groupby("firm_id").last().reset_index()
            slope_omit_col = f"{prefix}_slope_omit_{slope_type}"
            intercept_omit_col = f"{prefix}_intercept_omit_{slope_type}"
            df_final = df_final.merge(
                df_omit[["firm_id", slope_omit_col, intercept_omit_col]],
                on="firm_id", how="left"
            ).rename(columns={
                slope_omit_col: f"{rename_prefix}_growth_omit{y}_{slope_type}",
                intercept_omit_col: f"{rename_prefix}_intercept_omit{y}_{slope_type}"
            })

    return df_final


# --- Merge LINEAR slopes ---
df_final = merge_slopes_all(df_final, df_slopes_linear, prefix="elec_share")
df_final = merge_slopes_all(df_final, df_slopes_linear, prefix="low-carbon_elec_share")

# --- Merge EXPONENTIAL slopes ---
df_final = merge_slopes_all(df_final, df_slopes_exp, prefix="elec_share", rename_prefix="elec_share_exp")
df_final = merge_slopes_all(df_final, df_slopes_exp, prefix="low-carbon_elec_share", rename_prefix="lowcarbon_elec_share_exp")


In [102]:
df_final.to_csv("results/energy_consumption_and_trends_firms_2020-2024.csv",index=False)